This Jupyter notebook has been tested with Python 3.13.2 on macOS. Adjust PostgreSQL paths and Docker credentials if necessary.

# Setup #

In [1]:
from IPython.core.magic import register_line_cell_magic
import re
REPO_HTTPS_URL = 'https://github.com/KCony/Farmers-Markets.git'
WORKSPACE_HOME = './workspace/'
NAME_PATTERN = r'.*/(.+)\.git'
name = re.search(NAME_PATTERN, REPO_HTTPS_URL).group(1)

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Clean up any existing workspace directory:

In [2]:
!rm -rf {WORKSPACE_HOME}

Create and move into workspace directory:

In [3]:
from pathlib import Path
Path(f"{WORKSPACE_HOME}").mkdir(parents=True, exist_ok=True)
%cd {WORKSPACE_HOME}

/Users/jordankrishnayah/Downloads/workspace


Clone GitHub repository:

In [4]:
!git clone "{REPO_HTTPS_URL}" "{name}"
%cd {name}

Cloning into 'Farmers-Markets'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.(24/57)
remote: Compressing objects: 100% (40/40), done.
remote: Total 57 (delta 14), reused 51 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 215.12 KiB | 5.66 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/Users/jordankrishnayah/Downloads/workspace/Farmers-Markets


# Optional: Local Django Test Setup (with PostgreSQL)

In [5]:
# PostgreSQL settings for local macOS instance
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_USER = 'db_username'
DB_PASSWORD = 'db_password'
DB_NAME = 'db_name'
EXISTING_DB_NAME = 'postgres'

# Create new database
!echo "CREATE DATABASE {DB_NAME};" | psql postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{EXISTING_DB_NAME}

# Load SQL schema
!psql postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME} < farmers_markets.sql

psql: error: connection to server at "localhost" (::1), port 5432 failed: FATAL:  role "db_username" does not exist
psql: error: connection to server at "localhost" (::1), port 5432 failed: FATAL:  role "db_username" does not exist


Update .env file with your PostgreSQL settings:

In [6]:
%%writetemplate .env
# PostgreSQL Database Settings

DB_ENGINE=django.db.backends.postgresql
DB_NAME={DB_NAME}
DB_USER={DB_USER}
DB_PASSWORD={DB_PASSWORD}
DB_HOST={DB_HOST}
DB_PORT={DB_PORT}

Create and activate a virtual environment:

In [9]:
!python3 -m venv venv
source venv/bin/activate

SyntaxError: invalid syntax (3356957266.py, line 2)

Install dependencies:

In [10]:
!venv/bin/python -m pip install -r requirements.txt --disable-pip-version-check

zsh:1: no such file or directory: venv/bin/python


Run Django development server:

In [11]:
import subprocess
command = ['./venv/bin/python', 'manage.py', 'runserver']
process = subprocess.Popen(command)

FileNotFoundError: [Errno 2] No such file or directory: './venv/bin/python'

Open Django site locally:

In [ ]:
import webbrowser
webbrowser.open('http://localhost:8000/farmers_markets')

# Docker Deployment (macOS compatible)

In [12]:
DOCKER_USERNAME = 'imjordannyc'
DB_PORT = '5432'
DB_USER = 'db_username'
DB_PASSWORD = 'db_password'
DB_NAME = 'db_name'

!mkdir -p init-scripts
!cp farmers_markets.sql ./init-scripts/

!docker --version

Docker version 28.5.1, build e180ab8


In [13]:
%%writefile .dockerignore
venv
.git
.gitignore
__pycache__

Overwriting .dockerignore


In [14]:
%%writefile Dockerfile
FROM python:3.13-slim-bullseye
ENV PYTHONUNBUFFERED=1 PYTHONDONTWRITEBYTECODE=1 PIP_DISABLE_PIP_VERSION_CHECK=1
WORKDIR /app
COPY ./requirements.txt /app/
RUN pip install -r requirements.txt
COPY ./ /app/
EXPOSE 8000
CMD ["python", "manage.py", "runserver", "0.0.0.0:8000"]

Writing Dockerfile


In [15]:
%%writetemplate docker-compose.yml
services:
  db:
    image: postgres:17-alpine
    container_name: farmers_markets_db
    volumes:
      - ./postgres_data:/var/lib/postgresql/data/
      - ./init-scripts:/docker-entrypoint-initdb.d
    ports:
      - "5432:5432"
    environment:
      POSTGRES_DB: ${DB_NAME}
      POSTGRES_USER: ${DB_USER}
      POSTGRES_PASSWORD: ${DB_PASSWORD}
    restart: always

  web:
    image: ${DOCKER_USERNAME}/farmers_markets_app:latest
    build: .
    container_name: farmers_markets_app
    command: python manage.py runserver 0.0.0.0:8000
    volumes:
      - .:/app
    ports:
      - "8000:8000"
    depends_on:
      - db
    env_file:
      - ./.env

volumes:
  postgres_data:

In [26]:
%%writetemplate .env
# PostgreSQL Database Settings
DB_ENGINE=django.db.backends.postgresql
DB_NAME={DB_NAME}
DB_USER={DB_USER}
DB_PASSWORD={DB_PASSWORD}
DB_HOST={DB_HOST}
DB_PORT={DB_PORT}


In [20]:
!docker compose down --rmi all -v --remove-orphans
!docker compose up -d --build

WARN[0000] The "db_name" variable is not set. Defaulting to a blank string. 
WARN[0000] The "db_username" variable is not set. Defaulting to a blank string. 
WARN[0000] The "db_password" variable is not set. Defaulting to a blank string. 
WARN[0000] The "imjordannyc" variable is not set. Defaulting to a blank string. 
[+] Running 0/1
 ⠋ Image postgres:17-alpine  Removing                                      0.0s 
[+] Running 1/1
 ✔ Image postgres:17-alpine  Removed                                       0.1s 
WARN[0000] The "db_name" variable is not set. Defaulting to a blank string. 
WARN[0000] The "db_username" variable is not set. Defaulting to a blank string. 
WARN[0000] The "db_password" variable is not set. Defaulting to a blank string. 
WARN[0000] The "imjordannyc" variable is not set. Defaulting to a blank string. 
[+] Running 0/1
 ⠙ db Pulling                                                              0.1s 
[+] Running 0/1
 ⠹ db Pulling                                        

In [11]:
import webbrowser
webbrowser.open('http://localhost:8000/farmers_markets')

True

In [ ]:
!docker login
!docker compose push

In [22]:
# --- Define global variables ---
DOCKER_USERNAME = 'imjordannyc'
DB_NAME = 'farmers_db'
DB_USER = 'farm_user'
DB_PASSWORD = 'supersecret'
DB_HOST = 'db'
DB_PORT = '5432'



In [24]:
# --- Write .env ---
%%writetemplate .env
DB_ENGINE=django.db.backends.postgresql
DB_NAME={DB_NAME}
DB_USER={DB_USER}
DB_PASSWORD={DB_PASSWORD}
DB_HOST={DB_HOST}
DB_PORT={DB_PORT}


UsageError: Line magic function `%%writetemplate` not found.


In [27]:
%%writetemplate docker-compose.yml
version: '3.9'
services:
  db:
    image: postgres:17-alpine
    container_name: farmers_markets_db
    environment:
      POSTGRES_DB: ${DB_NAME}
      POSTGRES_USER: ${DB_USER}
      POSTGRES_PASSWORD: ${DB_PASSWORD}
    volumes:
      - ./postgres_data:/var/lib/postgresql/data/
      - ./init-scripts:/docker-entrypoint-initdb.d
    ports:
      - "5432:5432"
    restart: always

  web:
    build: .
    image: {DOCKER_USERNAME}/farmers_markets_app:latest
    container_name: farmers_markets_app
    command: python manage.py runserver 0.0.0.0:8000
    volumes:
      - .:/app
    ports:
      - "8000:8000"
    depends_on:
      - db
    env_file:
      - ./.env

volumes:
  postgres_data:

In [28]:
!docker compose down -v --remove-orphans
!docker compose up -d --build

WARN[0000] The "farmers_db" variable is not set. Defaulting to a blank string. 
WARN[0000] The "farm_user" variable is not set. Defaulting to a blank string. 
WARN[0000] The "supersecret" variable is not set. Defaulting to a blank string. 
WARN[0000] /Users/jordankrishnayah/Downloads/workspace/Farmers-Markets/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
WARN[0000] The "farmers_db" variable is not set. Defaulting to a blank string. 
WARN[0000] The "farm_user" variable is not set. Defaulting to a blank string. 
WARN[0000] The "supersecret" variable is not set. Defaulting to a blank string. 
WARN[0000] /Users/jordankrishnayah/Downloads/workspace/Farmers-Markets/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Building 0.0s (0/1)                                                         
 => [internal] load local bake definitions       